In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

In [0]:
import sys
sys.path.append('/Workspace/Users/bjedelma@gmail.com/ScaleML/scaleml')
import scaleml


Detect computational resources on current node

In [0]:
# Detect computing resources
detected_resources = scaleml.resources()

In [0]:
# Specify ML frameworks for the model and distributed strategy
framework = {
    "model": "tensorflow",
    "strategy": "tensorflow",
}

# Set the distributed training strategy
dist_strategy = scaleml.strategies(framework, detected_resources, resource_type='all')

In [0]:
# Load MNIST dataset for example
import tensorflow as tf

# MNIST as example model and dataset (you can replace with your actual dataset/model)
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the images to a range of 0 to 1
train_images, test_images = train_images / 255.0, test_images / 255.0

# Select half the train data
half_index = len(train_images) // 8
train_images, train_labels = train_images[:half_index], train_labels[:half_index]
train_images = train_images.reshape((-1, 28, 28, 1))

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).batch(64)

display(train_images, train_labels, test_images, test_labels)

In [0]:
print(f"Train images shape: {train_images.shape}")
print(f"Train labels shape: {train_labels.shape}")

In [0]:
dist_strategy.scope

In [0]:
# Train model with distributed strategy
history, history_file, log_file = scaleml.distributed_train(framework, dist_strategy, train_dataset, log_resources=True)

In [0]:
# def distributed_train(framework, strategy, train_dataset, log_resources=True):
strategy = dist_strategy
"""
Train a model using a specified framework and strategy, and track resource usage (optional).

Parameters:
- framework (dict): A dictionary containing the framework details.
                        Expected keys: 'model', 'strategy'.
                        Options for 'model': 'tensorflow', 'pytorch'.
                        Options for 'strategy': 'tensorflow', 'pytorch', 'horovod'.
- strategy: distributed ScaleML training strategy appropriate for the model.
- train_dataset: Dataset used for training.
- log_resources: Boolean flag to enable/disable resource logging.
"""

import tensorflow as tf
import torch
from datetime import datetime
from multiprocessing import Process, Value
import os, time, pickle
# import horovod.tensorflow as hvd_tf
# import horovod.torch as hvd_torch

from scaleml import create_scaleml_folders, create_model, start_logging, stop_logging

# create folders for saving
scaleml_folder = create_scaleml_folders()
print(f"Scaleml folder and subfolders are set up at: {scaleml_folder}")
print()

# log resource use throughout training
log_dir = f"{scaleml_folder}/logs/"
log_file = f"{log_dir}/{datetime.now().strftime('%Y%m%d')}_resource_usage_log.csv"
# if log_resources:        
# Start logging resources in the background (adjust the interval as needed)
log_process = start_logging(log_file, interval=10)
print()

# Detect size of the first image from train_dataset
for images, labels in train_dataset.take(1):
    input_shape = images.shape[1:]
    break

# create model based on the detected framework
model = create_model(strategy, framework, input_shape)

# Save the model
log_time = log_file.split('/')[-1]
log_time = log_time.split('_')[0]
if framework == 'tensorflow':
    model.save(os.path.join(f"{scaleml_folder}/models/", f"{log_time}_tf_model.h5"))
elif framework == 'pytorch':
    torch.save(model.state_dict(), os.path.join(f"{scaleml_folder}/models/", 'f"{log_time}_torch_model.pth'))

# Start the training process with or without resource monitoring
if isinstance(strategy, tf.distribute.MirroredStrategy):

    with strategy.scope():
        history = model.fit(train_dataset, epochs=2)
        history_file = os.path.join(f"{scaleml_folder}/models/", f"{log_time}_tf_model_history.pkl")
        with open(history_file, 'wb') as f:
            pickle.dump(history.history, f)

elif isinstance(strategy, torch.device):
    model.to(strategy)
    # history = train_pytorch_model(model, train_dataset, epochs=2)
    history_file = os.path.join(f"{scaleml_folder}/models/", f"{log_time}_torch_model_history.pkl")
    with open(history_file, 'wb') as f:
            pickle.dump(history.history, f)

# elif 'horovod' in str(type(strategy)).lower():

# # if log_resources:
# # Stop the logging once training is complete
# stop_logging(log_process)

# print("Training completed.")
# return history, history_file, log_file